<center><img src="MKn_Staffelter_Hof.jpeg" alt="Picture of old business"</center>
<!--Image Credit: Martin Kraft https://commons.wikimedia.org/wiki/File:MKn_Staffelter_Hof.jpg -->

Staffelter Hof Winery is Germany's oldest business, established in 862 under the Carolingian dynasty. It has continued to serve customers through dramatic changes in Europe, such as the Holy Roman Empire, the Ottoman Empire, and both world wars. What characteristics enable a business to stand the test of time?

To help answer this question, BusinessFinancing.co.uk researched the oldest company still in business in **almost** every country and compiled the results into several CSV files. This dataset has been cleaned.

Having useful information in different files is a common problem. While it's better to keep different types of data separate for data storage, you'll want all the data in one place for analysis. You'll use joining and data manipulation to work with this data and better understand the world's oldest businesses.

## The Data
`data/businesses.csv` and `data/new_businesses.csv`
|Column|Description|
|------|-----------|
|`business`|Name of the business (varchar)|
|`year_founded`|Year the business was founded (int)|
|`category_code`|Code for the business category (varchar)|
|`country_code`|ISO 3166-1 three-letter country code (char)|

`data/countries.csv`
|Column|Description|
|------|-----------|
|`country_code`|ISO 3166-1 three-letter country code (varchar)|
|`country`|Name of the country (varchar)|
|`continent`|Name of the continent the country exists in (varchar)|

`data/categories.csv`
|Column|Description|
|------|-----------|
|`category_code`|Code for the business category (varchar)|
|`category`|Description of the business category (varchar)|

In [342]:
# Import necessary libraries
import pandas as pd

# Load the data
businesses = pd.read_csv("data/businesses.csv")
new_businesses = pd.read_csv("data/new_businesses.csv")
countries = pd.read_csv("data/countries.csv")
categories = pd.read_csv("data/categories.csv")

## Getting Started
To get started, I first printed the first few rows of each dataframe to get a better idea of what I was working with. I saw that both the `businesses` and `new_businesses` dataframes contain the same columns, so I joined them together into a new dataframe called `all_businesses` using `pd.concat`. I then checked the lengths of the columns to double-check that the concatenation worked. I then merged `all_businesses` with `countries` using a left join, and then merged them with `categories` using a left join to create the dataframe `df_all`. I chose left joins because the data I am most interested in is pertaining to the oldest buisnesses, and therefore I do not need any info in the `countries` or `categories` datasets if it is not related to a specific buisness. 

In [343]:
print(businesses.head())
print(new_businesses.head())
print(countries.head())
print(categories.head())

                         business  year_founded category_code country_code
0                  Hamoud Boualem          1878         CAT11          DZA
1  Communauté Électrique du Bénin          1968         CAT10          BEN
2        Botswana Meat Commission          1965          CAT1          BWA
3                     Air Burkina          1967          CAT2          BFA
4                         Brarudi          1955          CAT9          BDI
                    business  year_founded category_code country_code
0                 Fiji Times          1869         CAT13          FJI
1  J. Armando Bermúdez & Co.          1852          CAT9          DOM
  country_code               country continent
0          AFG           Afghanistan      Asia
1          AGO                Angola    Africa
2          ALB               Albania    Europe
3          AND               Andorra    Europe
4          ARE  United Arab Emirates      Asia
  category_code                   category
0          CAT1

In [344]:
all_businesses = pd.concat([businesses, new_businesses], ignore_index=True)
print(len(businesses) + len(new_businesses) == len(all_businesses))


True


In [345]:
businesses_countries = all_businesses.merge(countries, on='country_code', how='left')
businesses_countries.head()


,business,year_founded,category_code,country_code,country,continent
0,Hamoud Boualem,1878,CAT11,DZA,Algeria,Africa
1,Communauté Électrique du Bénin,1968,CAT10,BEN,Benin,Africa
2,Botswana Meat Commission,1965,CAT1,BWA,Botswana,Africa
3,Air Burkina,1967,CAT2,BFA,Burkina Faso,Africa
4,Brarudi,1955,CAT9,BDI,Burundi,Africa


In [346]:
df = businesses_countries.merge(categories, on='category_code', how='left')
df.head()

,business,year_founded,category_code,country_code,country,continent,category
0,Hamoud Boualem,1878,CAT11,DZA,Algeria,Africa,Food & Beverages
1,Communauté Électrique du Bénin,1968,CAT10,BEN,Benin,Africa,Energy
2,Botswana Meat Commission,1965,CAT1,BWA,Botswana,Africa,Agriculture
3,Air Burkina,1967,CAT2,BFA,Burkina Faso,Africa,Aviation & Transport
4,Brarudi,1955,CAT9,BDI,Burundi,Africa,"Distillers, Vintners, & Breweries"


## Question 1: What is the oldest business on each continent? Save your answer as a DataFrame called `oldest_business_continent` with four columns: `continent`, `country`, `business`, and `year_founded` in any order.
I first created a shortened version of my main DataFrame that contained only the columns of interest in the specified order, saving it as `df_small_reordered`. To find the oldest buisness year in each continent, I used `.groupby` to group by dataset by 'continent', and then ran `.idxmax` on the 'year_founded' column to find the index of the entry that contained the oldest business year in each continent. I used `.idxmax` to get the index because I could then put the indices into `.loc` for indexing to be able to print out the entire row of data for each of the oldest years per continent. Had I used `.min` instead of `.idxmin` and not indexed using `.loc`, I only would have received a pandas series containing the continent and the oldest year, not the entire row of data. 

In [347]:
df_small_reordered = df[['continent', 'country', 'business', 'year_founded']]
oldest_business_continent = df_small_reordered.loc[df_small_reordered.groupby('continent')['year_founded'].idxmin()]
print(oldest_business_continent)

         continent    country                     business  year_founded
29          Africa  Mauritius               Mauritius Post          1772
64            Asia      Japan                   Kongō Gumi           578
94          Europe    Austria  St. Peter Stifts Kulinarium           803
145  North America     Mexico  La Casa de Moneda de México          1534
160        Oceania  Australia               Australia Post          1809
157  South America       Peru      Casa Nacional de Moneda          1565


## Question 2: How many countries per continent lack data on the oldest businesses? Does including `new_businesses` change this? Count the number of countries per continent missing business data, including `new_businesses`, and store the results in a DataFrame named `count_missing` with columns for the continent and the count.
I first found the count of countries per continent lacking data on their oldest buisiness in the `business` DataFrame. To do so, I merged the `businesses` and `countries` dataframes in a new dataframe called `business_country`. I then subset `business_country` to only contain rows in which the value in the '_merge' column was not equal to 'both', indicating that the country is not in the `business` dataset and therefore is missing data on its oldest business. I saved the newly subset DataFrame as `count_not_in_both`. I then counted the number of countries per continent missing data and saved it as the DataFrame `count_missing_pre`, renaming the column to 'count_missing'. To find the number of countries per continent lacking data on the oldest business across all countries we have data for, I repeated the process above, but included the data in `new_businesses`. Doing so revealed that the counts do change when including `new_businesses`, with Oceania and North America each gaining an additional country they have data for. 

In [348]:
business_country = businesses.merge(countries, on= 'country_code', how = 'outer', indicator=True)
count_not_in_both = business_country[business_country['_merge'] != 'both']
count_missing_pre = pd.DataFrame(count_not_in_both.groupby('continent').agg({'country': 'count'}))
count_missing_pre.columns = ['count_missing']
print(count_missing_pre)

               count_missing
continent                   
Africa                     3
Asia                       7
Europe                     2
North America              6
Oceania                   11
South America              3


In [349]:
all_business_country = all_businesses.merge(countries, on= 'country_code', how = 'outer', indicator=True)
all_count_not_in_both = all_business_country[all_business_country['_merge'] != 'both']
count_missing = pd.DataFrame(all_count_not_in_both.groupby('continent').agg({'country': 'count'}))
count_missing.columns = ['count_missing']
print(count_missing)
print(count_missing == count_missing_pre)


               count_missing
continent                   
Africa                     3
Asia                       7
Europe                     2
North America              5
Oceania                   10
South America              3
               count_missing
continent                   
Africa                  True
Asia                    True
Europe                  True
North America          False
Oceania                False
South America           True


In [350]:
print(count_missing)

               count_missing
continent                   
Africa                     3
Asia                       7
Europe                     2
North America              5
Oceania                   10
South America              3


## Question 3: Which business categories are best suited to last many years, and on what continent are they? Create a DataFrame called `oldest_by_continent_category` that stores the oldest founding year for each continent and category combination. It should contain three columns: `continent`, `category`, and `year_founded`, in that order.
I first merged my DataFrame that contained the content from `businesses` and `countries` with the DataFrame `categories`, using a left join. I then grouped the data by continent and category and found the minumum year for each continent and category combination, storing the data in the DataFrame `oldest_by_continent_category`.

In [351]:
bus_count_cat = business_country.merge(categories, on='category_code', how='left')
oldest_by_continent_category = bus_count_cat.groupby(['continent', 'category']).agg({'year_founded': 'min'})
print(oldest_by_continent_category)
print(oldest_by_continent_category.shape)

                                                 year_founded
continent     category                                       
Africa        Agriculture                              1947.0
              Aviation & Transport                     1854.0
              Banking & Finance                        1892.0
              Distillers, Vintners, & Breweries        1933.0
              Energy                                   1968.0
              Food & Beverages                         1878.0
              Manufacturing & Production               1820.0
              Media                                    1943.0
              Mining                                   1962.0
              Postal Service                           1772.0
Asia          Agriculture                              1930.0
              Aviation & Transport                     1858.0
              Banking & Finance                        1830.0
              Cafés, Restaurants & Bars                1153.0
        